# 0. Necessary Gatherings

In [1]:
!pip install tqdm
import numpy as np
import pandas as pd 

from tqdm import tqdm
from transformers import pipeline
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from IPython.display import clear_output

2024-02-29 09:17:36.736916: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-29 09:17:36.737026: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-29 09:17:36.860618: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
import torch
torch_device = torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 0.1 Give Test Dataset in CSV [3 Columns : Index District Contents] [Just Change Here To Infer With Private Dataset, Don't Need to change anywhere else]

In [3]:
DATASET_CSV = '/kaggle/input/regipa/test_regipa.csv'


# 0.2 Give Model Directory/ Hugging Face Model id [Don't change here]

In [4]:
MODEL_DIRECTORY = 'abdullaharean/regipa_bangla'

# 1. Preprocessing

In [5]:
test_df= pd.read_csv(DATASET_CSV)
# test_df.tail(10)

## 1.1 Split Cells With Multiple Lines

In [6]:
def split_lines(row):
    lines = row['Contents'].split('\n')
    index = [row.name] * len(lines)  # Duplicate index for each line
    return pd.DataFrame({'Index': index, 'District': row['District'], 'Contents': lines})

test_df = pd.concat([split_lines(row) for _, row in test_df.iterrows()], ignore_index=True)
# test_df.tail(10)

## 1.2 Add Punctuation Where Not

In [7]:
punctuation_marks = [',', '?', '।', '!','"']
indices_to_append_period = test_df[~test_df['Contents'].str.endswith(tuple(punctuation_marks))].index
test_df.loc[indices_to_append_period, 'Contents'] += '।'
# test_df.tail(10)

In [8]:
# indices_to_append_period

# 2. Output From Model

Our trained model is uploaded in hugging-face can be accessed through this link: https://huggingface.co/abdullaharean/regipa_bangla

In [9]:
model_directory = MODEL_DIRECTORY
tokenizer = AutoTokenizer.from_pretrained(model_directory)
model = AutoModelForSeq2SeqLM.from_pretrained(model_directory)
torch_device='cuda'
model = model.to(torch_device)
model = model.eval()

tokenizer_config.json:   0%|          | 0.00/25.6k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/3.02k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.09k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/837 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

In [10]:
def calc_token_len(example):
    return len(tokenizer(example).input_ids)

def ipa_converter(input_texts, num_return_sequences, input_len):
    input_len = max(input_len)
    batch = tokenizer(input_texts, truncation=True, padding='max_length', max_length=input_len, return_tensors="pt").to(torch_device)
    translated = model.generate(**batch, max_length=input_len, num_beams=4, num_return_sequences=num_return_sequences, temperature=1.5)
    tgt_texts = tokenizer.batch_decode(translated, skip_special_tokens=True)
    
    return tgt_texts


In [11]:
test_df['input_token_len'] = test_df['Contents'].apply(calc_token_len)
texts = test_df["Contents"].tolist()
lengths = test_df["input_token_len"].tolist()
input_data = list(zip(texts, lengths))

ipa_texts = []
batch_size = 64  # You can adjust the batch size based on your system's memory constraints

for i in tqdm(range(0, len(input_data), batch_size)):
    batch = input_data[i:i+batch_size]
    batch_texts, batch_lengths = zip(*batch)  # Unpack the tuples
    
    # Ensure that batch_lengths are integers
    batch_lengths = list(map(int, batch_lengths))
    
    ipa_batch = ipa_converter(batch_texts, 1, batch_lengths)
#     print(ipa_batch)
    ipa_texts.extend(ipa_batch)
test_df.drop(columns=['input_token_len'], inplace=True)

  0%|          | 0/142 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
100%|██████████| 142/142 [22:00<00:00,  9.30s/it]


# 3. Postproessing

In [12]:
# ipa_texts

In [13]:
result_df = test_df.copy()
result_df['string']= ipa_texts
# result_df.tail(10)

## 3.1 Post Process Remove '|'

In [14]:
result_df.loc[indices_to_append_period, 'string'] = result_df.loc[indices_to_append_period, 'string'].str.rstrip('।')
# result_df.tail(10)

## 3.2 Group Splitted Cells

In [15]:
result_df = result_df.groupby(['Index', 'District'])['string'].apply('\n'.join).reset_index()
result_df['id'] = result_df['Index']
result_df.drop(columns=['Index', 'District'], inplace=True)
result_df = result_df[['id', 'string']]
# result_df.tail(10)

## 3.3 Rectify Errors Around Punctuation (Working Like a charm)

In [16]:
def word_count(sentence):
    words = sentence.split()
    return len(words)
def find_and_get_neighbors(s, symbol):
    index = s.find(symbol)
    if index != -1:
        before = s[index - 1] if index - 1 >= 0 else None
        after = s[index + 1] if index + 1 < len(s) else None
        return before, after, index
    else:
        return None, None, None
def rewrite_sentence(sentence, final):
    output = ""
    list0 = [',', '?', '<>', '।', '!','"']
    list1 = [' ', '\n']
    final_copy = final  # Make a copy to manipulate
    for char in sentence:
        if char in list0:
            before, after, index = find_and_get_neighbors(final_copy, char)
            if before in list1 and not final_copy.endswith(before):  # Corrected syntax here
                output += before
            output += char
            if after in list1:
                output += after
            if index is not None:
                final_copy = final_copy[:index-1] + final_copy[index+1:]  # Remove char and its preceding character
        else:
            if output.endswith(char) and char in list1:
                pass
            else:
                output += char
    return output
def special_postprocess1(df, test_df):
    temp_df = df.copy()
    
    temp_df['Contents']=test_df['Contents']
    temp_df['Count_Contents'] = temp_df['Contents'].apply(lambda x: word_count(x))
    temp_df['Count_string'] = temp_df['string'].apply(lambda x: word_count(x))
    
    unequal_rows = temp_df[temp_df['Count_Contents'] != temp_df['Count_string']]
    print(f"PostProcessing1: There are {len(unequal_rows)} rows having different word count in Content and IPA!\nRectifying...")
    
    for index, row in unequal_rows.iterrows():
        idtochange = row['id']
        temp_df.loc[idtochange, 'string'] = rewrite_sentence(temp_df.loc[idtochange, 'string'], temp_df.loc[row['id'], 'Contents'])
    
    temp_df.drop(columns=['Contents','Count_Contents','Count_string'], inplace=True)
    return temp_df


## 3.4 Word By Word IPA (Still Not Improving Score)

In [17]:
def word_to_ipa(input_text,model_directory=MODEL_DIRECTORY):
    tokenizer = AutoTokenizer.from_pretrained(model_directory)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_directory)
    torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = model.to(torch_device)
    model = model.eval()
    
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(torch_device)
    ipa_sequences = model.generate(input_ids)
    ipa_texts = tokenizer.batch_decode(ipa_sequences, skip_special_tokens=True)
    ipa = ipa_texts[0] if ipa_texts else None
    
    return ipa
def bangla_to_ipa(text, model_directory=MODEL_DIRECTORY):
    print(text)
    lines = text.splitlines()
    ipa_lines = []

    for line in lines:
        words = line.split()
        ipa_text = ''
        special_characters = [',', '?', '<>', '।', '!', ' ', '\n']
        
        for word in words:
            if word in special_characters:
                ipa_text += word
            else:
                ipa_text += word_to_ipa(word, model_directory)
            ipa_text += " "

        ipa_lines.append(ipa_text.rstrip())  # Remove trailing whitespace

    return '\n'.join(ipa_lines)
def special_postprocess2(df, test_df,model_directory=MODEL_DIRECTORY):
    final_df = df.copy()
    final_df['Contents'] = test_df['Contents']
    final_df['District'] = test_df['District']
    final_df['Count_Contents'] = final_df['Contents'].apply(word_count)
    final_df['Count_string'] = final_df['string'].apply(word_count)
    unequal_rows = final_df[final_df['Count_Contents'] != final_df['Count_string']]
    index = 0
    print(f"PostProcessing1: There are {len(unequal_rows)} rows having different word count in Content and IPA!\nRectifying...")
    for _, row in unequal_rows.iterrows():
        index += 1
        final_df.loc[row['id'], 'string'] = bangla_to_ipa(final_df.loc[row['id'], 'Contents'], model_directory)
        print(final_df.at[row['id'], 'string'])
    final_df.drop(columns=['Contents','Count_Contents','Count_string','District'], inplace=True)
    return final_df

# 4. Submission

In [18]:
# result_df.to_csv('submission.csv', index=False)
dataset_csv = pd.read_csv(DATASET_CSV)
pp1_result_df = special_postprocess1(result_df, dataset_csv)
pp1_result_df.to_csv('team_khita_kortesi_submission.csv', index=False)

# pp2_result_df = special_postprocess2(pp1_result_df, dataset_csv,MODEL_DIRECTORY)
# pp2_result_df.to_csv('submission_pp2.csv', index=False)

clear_output()

In [19]:
# result_df.tail(10)

In [20]:
# pp1_result_df.tail(10)

In [21]:
# pp2_result_df.tail(10)